In [14]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error 
import matplotlib.pyplot as plt
import numpy as np
import math
import time
from scipy.stats import randint
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
excel_file1 = 'C:\\Users\\satya\\OneDrive\\Desktop\\cleans.xlsx'
df = pd.read_excel(excel_file1)
csv_file = 'stry.csv'  
df.to_csv('stry.csv', index=False)


# Label Encoding for categorical columns
label_encoder = LabelEncoder()
df['Payment Mode'] = label_encoder.fit_transform(df['Payment Mode'])
# Transform 'Segment', 'Region', 'Ship Mode' columns
df['Segment'] = label_encoder.fit_transform(df['Segment'])
df['Region'] = label_encoder.fit_transform(df['Region'])
df['Ship Mode'] = label_encoder.fit_transform(df['Ship Mode'])

df['Order Date'] = pd.to_datetime(df['Order Date'], format='%m-%d-%Y')

df2019 = df[df['Order Date'].dt.year == 2019]
df2020 = df[df['Order Date'].dt.year == 2020]

X = df2019[['Region', 'Segment', 'Ship Mode','Quantity','Payment Mode']]
y = df2019['Sales']


df2020.drop('Sales',axis=1,inplace=True)
start_time_training = time.time()

# Assuming 'X' is the quantity feature and 'y' is the corresponding profits
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_trainre=y_train.values.reshape(-1,1)
y_testre=y_test.values.reshape(-1,1)
param_dist = {
    'n_estimators': randint(50, 500),  # Randomly sample between 50 to 500 estimators
    'max_depth': randint(2, 20),       # Randomly sample max depth between 2 to 20
    'min_samples_split': randint(2, 10), # Randomly sample min samples split between 2 to 10
    # Add other hyperparameters as needed
}

# Create a Random Forest Regressor
rf = RandomForestRegressor(random_state=42)

# Perform Randomized Search Cross Validation
random_search = RandomizedSearchCV(rf, param_distributions=param_dist, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = random_search.best_params_
best_estimator = random_search.best_estimator_

# Evaluate the best estimator on the test set
y_pred = best_estimator.predict(X_test)
mse = mean_squared_error(y_testre, y_pred)
print("Best Parameters:", best_params)
print("Best Estimator MSE:", mse)

# Creating a Decision Tree Regressor
model = RandomForestRegressor(n_estimators=363,max_depth=5)  # Example: setting max depth to 5

# Training the model
model.fit(X_train, y_trainre)
end_time_training = time.time()
training_time = end_time_training - start_time_training
useful=df2020[['Region', 'Segment', 'Ship Mode','Quantity','Payment Mode']]


start_time_predictions = time.time()# Making predictions
predictions = model.predict(X_test)

end_time_predictions = time.time()
prediction_time = end_time_predictions - start_time_predictions
mse = mean_squared_error(y_testre, predictions)
print(f"Mean Squared Error: {mse}")
print(math.sqrt(mse))

predictions1 = model.predict(useful)
print(predictions1)

print(f"Time taken for training: {training_time} seconds")
print(f"Time taken for predictions: {prediction_time} seconds")
x_values_scatter = df2019['Sales'].values
y_values_scatter = df2019['Quantity'].values
x_values_line = predictions1
y_values_line = df2020['Quantity'].values
# Create a figure and axis
fig, ax = plt.subplots()

# Plotting scatter plot
ax.scatter(x_values_scatter, y_values_scatter, label='Scatter', color='red')

# Plotting line plot
ax.plot(x_values_line, y_values_line, label='Line', color='blue')

# Set labels and title
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_title('Scatter and Line Plot')

# Show legend
ax.legend()

# Show the plot
plt.show()



C:\Users\satya\AppData\Local\Temp\ipykernel_4216\2504190364.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2020.drop('Sales',axis=1,inplace=True)


Best Parameters: {'max_depth': 2, 'min_samples_split': 5, 'n_estimators': 363}
Best Estimator MSE: 331550.7451882327


c:\Python311\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Mean Squared Error: 336184.5585798244
579.8142448921244
[258.14669183 242.22581702 265.17596662 ... 176.25615812 188.39374146
 198.00475815]
Time taken for training: 18.522085428237915 seconds
Time taken for predictions: 0.013877153396606445 seconds


NameError: name 'plt' is not defined